<a href="https://colab.research.google.com/github/tayfununal/PyTorch/blob/main/NN_with_pyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np

!pip install playground-data
import plygdata as pg

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
class Model(nn.Module):

  def __init__(self, input_dim, hidden_1, hidden_2, output_dim):
    super(Model, self).__init__()
    
    self.linear_1 = nn.Linear(input_dim, hidden_1)
    self.linear_2 = nn.Linear(hidden_1, hidden_2)
    self.linear_3 = nn.Linear(hidden_2, output_dim)

  def forward(self, x):
    x = torch.relu(self.linear_1(x))
    x = torch.relu(self.linear_2(x))
    x = torch.sigmoid(self.linear_3(x))
    return x

In [ ]:
model = Model(input_dim=2, hidden_1=32, hidden_2=32, output_dim=1).to(device)
model

In [ ]:
df = pd.DataFrame(pg.dataset.DataGenerator.classify_xor(1000, noise=0.1),columns=["x1", "x2", "y"])

In [ ]:
X = df[["x1","x2"]].to_numpy()
y = df[["y"]].to_numpy()

from torch.utils.data import TensorDataset

train_ds = TensorDataset(X, y)
torch.manual_seed(1)
batch_size = 2
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 100
loss_hist = [0] * num_epochs
accuracy_hist = [0] * num_epochs
for epoch in range(num_epochs):
  for x_batch, y_batch in train_dl:
    pred = model(x_batch)
    loss = loss_fn(pred, y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    loss_hist[epoch] += loss.item()*y_batch.size(0)
    is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
    accuracy_hist[epoch] += is_correct.mean()
    loss_hist[epoch] /= len(train_dl.dataset)
    accuracy_hist[epoch] /= len(train_dl.dataset)